In [1]:
%reload_ext autoreload
%autoreload 2

import os
import logging
import multiprocessing

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import arviz as az
import jax
import jax.numpy as jnp
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)


In [2]:
import numpyro.distributions as dist
from hbmep.model import BaseModel

class MixedEffects(BaseModel):
    Name = "mixed_effects_injury_threshold_comparison"

    def __init__(self, config: Config):
        super(MixedEffects, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def fn(self, x, a, b, v, L, l, H):
        return (
            L
            + jnp.where(
                jnp.less(x, a),
                0.,
                -l + jnp.true_divide(
                    H + l,
                    jnp.power(
                        1
                        + jnp.multiply(
                            -1
                            + jnp.power(
                                jnp.true_divide(H + l, l),
                                v
                            ),
                            jnp.exp(jnp.multiply(-b, x - a))
                        ),
                        jnp.true_divide(1, v)
                    )
                )
            )
        )

    def _model(self, subject, features, intensity, response_obs=None):
        subject, n_subject = subject
        features, n_features = features
        intensity, n_data = intensity

        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)
        n_feature0 = n_features[0]
        n_baseline = 1

        mask = np.full((n_subject, n_feature0, self.n_response), True)
        if response_obs is not None:
            mask = np.full((n_subject, n_feature0, self.n_response), False)
            combinations = list(set(zip(subject.tolist(), feature0.tolist())))
            for c in combinations:
                mask[*c, :] = True

        """ Fixed Effect (Baseline) """
        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_baseline", n_baseline, dim=-2):
                a_mean_baseline = numpyro.sample("a_mean_baseline", dist.HalfNormal(scale=50))

        """ Random Effect (Delta) """
        a_mean_delta_global_loc = numpyro.sample("a_mean_delta_global_loc", dist.Normal(loc=0, scale=100))
        a_mean_delta_global_scale = numpyro.sample("a_mean_delta_global_scale", dist.HalfNormal(100))
        a_mean_delta_global_df_minus_one = numpyro.sample("a_mean_delta_global_df_minus_one", dist.Exponential(1 / 29.))

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_delta", (n_feature0 - n_baseline), dim=-2):
                a_mean_delta = numpyro.sample(
                    "a_mean_delta",
                    dist.StudentT(
                        loc=a_mean_delta_global_loc,
                        scale=a_mean_delta_global_scale,
                        df=a_mean_delta_global_df_minus_one + 1
                    )
                )

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            """ Global Priors """
            b_scale_global_scale = numpyro.sample("b_scale_global_scale", dist.HalfNormal(100))
            v_scale_global_scale = numpyro.sample("v_scale_global_scale", dist.HalfNormal(100))

            L_scale_global_scale = numpyro.sample("L_scale_global_scale", dist.HalfNormal(1))
            l_scale_global_scale = numpyro.sample("l_scale_global_scale", dist.HalfNormal(100))
            H_scale_global_scale = numpyro.sample("H_scale_global_scale", dist.HalfNormal(5))

            g_1_scale_global_scale = numpyro.sample("g_1_scale_global_scale", dist.HalfNormal(100))
            g_2_scale_global_scale = numpyro.sample("g_2_scale_global_scale", dist.HalfNormal(100))

            a_shape = numpyro.sample("a_shape", dist.HalfNormal(scale=100))

            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                """ Hyper-priors """
                a_mean = numpyro.deterministic(
                    "a_mean",
                    jnp.concatenate([a_mean_baseline, a_mean_baseline + a_mean_delta], axis=0)
                )

                b_scale_raw = numpyro.sample("b_scale_raw", dist.HalfNormal(scale=1))
                b_scale = numpyro.deterministic("b_scale", jnp.multiply(b_scale_global_scale, b_scale_raw))

                v_scale_raw = numpyro.sample("v_scale_raw", dist.HalfNormal(scale=1))
                v_scale = numpyro.deterministic("v_scale", jnp.multiply(v_scale_global_scale, v_scale_raw))

                L_scale_raw = numpyro.sample("L_scale_raw", dist.HalfNormal(scale=1))
                L_scale = numpyro.deterministic("L_scale", jnp.multiply(L_scale_global_scale, L_scale_raw))

                l_scale_raw = numpyro.sample("l_scale_raw", dist.HalfNormal(scale=1))
                l_scale = numpyro.deterministic("sigma_l", jnp.multiply(l_scale_global_scale, l_scale_raw))

                H_scale_raw = numpyro.sample("H_scale_raw", dist.HalfNormal(scale=1))
                H_scale = numpyro.deterministic("H_scale", jnp.multiply(H_scale_global_scale, H_scale_raw))

                g_1_scale_raw = numpyro.sample("g_1_scale_raw", dist.HalfNormal(scale=1))
                g_1_scale = numpyro.deterministic("g_1_scale", jnp.multiply(g_1_scale_global_scale, g_1_scale_raw))

                g_2_scale_raw = numpyro.sample("g_2_scale_raw", dist.HalfNormal(scale=1))
                g_2_scale = numpyro.deterministic("g_2_scale", jnp.multiply(g_2_scale_global_scale, g_2_scale_raw))

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    with numpyro.handlers.mask(mask=mask):
                        """ Priors """
                        a_raw = numpyro.sample("a_raw", dist.Gamma(concentration=a_shape, rate=1))
                        a = numpyro.deterministic(site.a, jnp.true_divide(jnp.multiply(a_raw, a_mean), a_shape))

                        b_raw = numpyro.sample("b_raw", dist.HalfNormal(scale=1))
                        b = numpyro.deterministic(site.b, jnp.multiply(b_scale, b_raw))

                        v_raw = numpyro.sample("v_raw", dist.HalfNormal(scale=1))
                        v = numpyro.deterministic(site.v, jnp.multiply(v_scale, v_raw))

                        L_raw = numpyro.sample("L_raw", dist.HalfNormal(scale=1))
                        L = numpyro.deterministic(site.L, jnp.multiply(L_scale, L_raw))

                        l_raw = numpyro.sample("l_raw", dist.HalfNormal(scale=1))
                        l = numpyro.deterministic("l", jnp.multiply(l_scale, l_raw))

                        H_raw = numpyro.sample("H_raw", dist.HalfNormal(scale=1))
                        H = numpyro.deterministic(site.H, jnp.multiply(H_scale, H_raw))

                        g_1_raw = numpyro.sample("g_1_raw", dist.HalfCauchy(scale=1))
                        g_1 = numpyro.deterministic(site.g_1, jnp.multiply(g_1_scale, g_1_raw))

                        g_2_raw = numpyro.sample("g_2_raw", dist.HalfCauchy(scale=1))
                        g_2 = numpyro.deterministic(site.g_2, jnp.multiply(g_2_scale, g_2_raw))

        """ Penalty """
        a_mean_penalty_for_negative = (jnp.fabs(a_mean_baseline + a_mean_delta) - (a_mean_baseline + a_mean_delta))
        numpyro.factor("a_mean_penalty_for_negative", -a_mean_penalty_for_negative)

        a_penalty_for_negative = (jnp.fabs(a) - a)
        numpyro.factor("a_penalty_for_negative", -a_penalty_for_negative)

        """ Outlier Distribution """
        outlier_dist_shape = numpyro.sample("outlier_dist_loc", dist.HalfNormal(5))
        outlier_dist_rate = numpyro.sample("outlier_dist_scale", dist.HalfNormal(1))

        """ Mixture """
        if response_obs is not None:
            outlier_prob = numpyro.sample("outlier_prob", dist.Uniform(0.0, 0.2))
        else: # Turn off mixture when predicting
            outlier_prob = numpyro.deterministic("outlier_prob", 0.)

        mixing_distribution = dist.Categorical(probs=jnp.array([1 - outlier_prob, outlier_prob]))

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_data, n_data, dim=-2):
                """ Model """
                mu = numpyro.deterministic(
                    site.mu,
                    self.fn(
                        x=intensity,
                        a=a[subject, feature0],
                        b=b[subject, feature0],
                        v=v[subject, feature0],
                        L=L[subject, feature0],
                        l=l[subject, feature0],
                        H=H[subject, feature0]
                    )
                )
                beta = numpyro.deterministic(
                    site.beta,
                    g_1[subject, feature0] + jnp.true_divide(g_2[subject, feature0], mu)
                )

                """ Mixture """
                component_distributions = [
                    dist.Gamma(concentration=jnp.multiply(mu, beta), rate=beta),
                    dist.Gamma(concentration=outlier_dist_shape, rate=outlier_dist_rate)
                ]
                Mixture = dist.MixtureGeneral(
                    mixing_distribution=mixing_distribution,
                    component_distributions=component_distributions
                )

                """ Observation """
                numpyro.sample(
                    site.obs,
                    Mixture,
                    obs=response_obs
                )



In [3]:
toml_path = "/home/vishu/repos/hbmep-paper/configs/paper/tms/injury-threshold-comparison/all_muscles.toml"

config = Config(toml_path=toml_path)
config.BUILD_DIR = os.path.join(config.BUILD_DIR, "parent")

config.MCMC_PARAMS["num_warmup"] = 5000
config.MCMC_PARAMS["num_samples"] = 1000

model = MixedEffects(config=config)


2023-11-13 14:02:56,619 - hbmep.config - INFO - Verifying configuration ...
2023-11-13 14:02:56,620 - hbmep.config - INFO - Success!
2023-11-13 14:02:56,636 - hbmep.model.baseline - INFO - Initialized base_model


In [4]:
src = "/home/vishu/data/hbmep-processed/human/tms/data_pkpk_auc_proc-2023-10-27.csv"
df = pd.read_csv(src)

df[model.features[0]] = \
    df[model.features[0]].replace({
        "Uninjured": "01_Uninjured",
        "SCI": "02_SCI"
    })

df, encoder_dict = model.load(df=df)


2023-11-13 14:02:56,760 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/tms/injury-threshold-comparison/all-muscles/parent
2023-11-13 14:02:56,761 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/tms/injury-threshold-comparison/all-muscles/parent
2023-11-13 14:02:56,762 - hbmep.dataset.core - INFO - Processing data ...
2023-11-13 14:02:56,763 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [5]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-11-13 14:02:56,836 - hbmep.model.baseline - INFO - Running inference with base_model ...


  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

2023-11-13 15:40:27,392 - hbmep.utils.utils - INFO - func:run_inference took: 1 hr and 37 min


In [7]:
prediction_df = model.make_prediction_dataset(df=df)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)

model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)
model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2023-11-13 15:42:19,301 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.00 sec


2023-11-13 15:42:47,741 - hbmep.utils.utils - INFO - func:predict took: 28.44 sec
2023-11-13 15:42:47,743 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-11-13 15:43:07,097 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/injury-threshold-comparison/all-muscles/parent/recruitment_curves.pdf
2023-11-13 15:43:07,097 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 19.35 sec
2023-11-13 15:43:07,098 - hbmep.model.baseline - INFO - Rendering posterior predictive checks ...
2023-11-13 15:43:27,469 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/injury-threshold-comparison/all-muscles/parent/posterior_predictive_check.pdf
2023-11-13 15:43:27,469 - hbmep.utils.utils - INFO - func:predictive_checks_renderer took: 20.37 sec
2023-11-13 15:43:27,470 - hbmep.utils.utils - INFO - func:render_predictive_check took: 20.37 sec


In [9]:
a_mean_delta_global_loc = posterior_samples["a_mean_delta_global_loc"]
a_mean_delta_global_loc = np.array(a_mean_delta_global_loc)
a_mean_delta_global_loc.shape

(a_mean_delta_global_loc > 0).mean()

0.68625

In [10]:
a_mean_delta = posterior_samples["a_mean_delta"]
a_mean_delta = np.array(a_mean_delta)
a_mean_delta.shape


(4000, 1, 6)

In [12]:
(a_mean_delta > 0).mean(axis=(0, 1))


array([0.75   , 0.8655 , 0.88775, 0.866  , 0.6085 , 0.571  ])

In [11]:
a_mean = posterior_samples["a_mean"]
a_mean = np.array(a_mean)
a_mean.shape

(4000, 2, 6)

In [13]:
a = posterior_samples["a"]
a = np.array(a)
a.shape

(4000, 16, 2, 6)

In [14]:
a[:, :10, 0, :].mean(axis=(0, 1))

array([35.56757006, 38.85423088, 35.88567561, 36.1304792 , 41.14470775,
       41.85531159])

In [16]:
a_mean[:, 0, :].mean(axis=0)

array([38.06397869, 47.32847007, 39.29299607, 37.93556347, 48.07296612,
       41.07867822])

In [18]:
a[:, 10:, 1, :].mean(axis=(0, 1))


array([34.92432531, 50.84076899, 49.21022322, 40.13312827, 51.33980972,
       45.21576117])

In [19]:
a_mean[:, 1, :].mean(axis=0)


array([40.4912466 , 50.57409988, 45.34157702, 42.05290083, 54.20192036,
       43.19568592])